In [36]:
import pandas as pd
import numpy as np

# declare the file name to read and file name to export
FILE_IN  = "googleplaystore.csv"
FILE_OUT = "googleplaystore_clean.csv"

df = pd.read_csv(FILE_IN)
df.head(3)


,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up


In [37]:

n0 = len(df)

# drop duplicate row and keep the first one
df = df.drop_duplicates(keep="first").reset_index(drop=True)

print("Removed duplicates:", n0 - len(df))


Removed duplicates: 483


In [38]:
#remove duplicate app name

n0 = len(df)

df = df.drop_duplicates(subset="App", keep="first").reset_index(drop=True)

print("Removed exact duplicates:", n0 - len(df))

Removed exact duplicates: 698


In [39]:
# drop the rating , review, last update column
df = df[[
    "Category","Size","Installs","Type","Price",
    "Content Rating","Genres","Android Ver"
]].copy()

df.head(3)


,Category,Size,Installs,Type,Price,Content Rating,Genres,Android Ver
0,ART_AND_DESIGN,19M,"10,000+",Free,0,Everyone,Art & Design,4.0.3 and up
1,ART_AND_DESIGN,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,4.0.3 and up
2,ART_AND_DESIGN,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,4.0.3 and up


In [40]:
# tranform the Size to MB with 2 decimals
s = df["Size"].astype(str).str.strip()

with_varies = s.str.contains("Varies", case=False, na=False)
with_m      = s.str.endswith(("M","m"), na=False)
with_k      = s.str.endswith(("K","k"), na=False)

df["Size"] = np.nan

# 13M/13m to 13.0
df.loc[with_m, "Size"] = pd.to_numeric(s[with_m].str[:-1])

# 850K/850k to 0.85
df.loc[with_k, "Size"] = pd.to_numeric(s[with_k].str[:-1]) / 1000.0

# trasform to numeric
plain = ~with_m & ~with_k & ~with_varies
df.loc[plain, "Size"] = pd.to_numeric(s[plain])

# fill "Varies with device" with the mean
mean_size_mb = df["Size"].mean()
df.loc[with_varies, "Size"] = mean_size_mb

# round to 2 decimals
df["Size"] = df["Size"].round(2)

df[["Size"]].head(10)

,Size
0,19.0
1,14.0
2,8.7
3,25.0
4,2.8
5,5.6
6,19.0
7,29.0
8,33.0
9,3.1


In [41]:
# remove + and , for installs
df["Installs"] = df["Installs"].astype(str).str.replace("+", "", regex=False)
df["Installs"] = df["Installs"].astype(str).str.replace(",", "", regex=False)

df[["Installs"]].head(10)


,Installs
0,10000
1,500000
2,5000000
3,50000000
4,100000
5,50000
6,50000
7,1000000
8,1000000
9,10000


In [42]:
# remove $ for price
df["Price"]= df["Price"].astype(str).str.replace("$", "", regex=False)
df["Price"]= df["Price"].astype("float64")
df[["Price"]].head(10)


,Price
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
5,0.0
6,0.0
7,0.0
8,0.0
9,0.0


In [43]:
#convert the type that is Nan value, if price more than 0 price = paid, if no its free
mask = df["Type"].isna()
df.loc[mask & (df["Price"] > 0), "Type"] = "Paid"
df.loc[mask & (df["Price"] == 0), "Type"] = "Free"

In [44]:
s = df["Android Ver"].astype(str).str.strip()

#split the first value
first_token = s.str.split().str[0]
major_part  = first_token.str.split(".").str[0]

mask_num = major_part.str.isnumeric()

# just take the first split 
df.loc[mask_num, "Android Ver"] = major_part[mask_num].astype(int)

# get the average of the version
avg_major = int(round(df.loc[mask_num, "Android Ver"].astype(int).mean()))

# for nan value, place average version
df.loc[~mask_num, "Android Ver"] = avg_major

df["Android Ver"] = df["Android Ver"].astype("int64")


In [45]:
print(df.dtypes)
df.head(3)


Category           object
Size              float64
Installs           object
Type               object
Price             float64
Content Rating     object
Genres             object
Android Ver         int64
dtype: object


,Category,Size,Installs,Type,Price,Content Rating,Genres,Android Ver
0,ART_AND_DESIGN,19.0,10000,Free,0.0,Everyone,Art & Design,4
1,ART_AND_DESIGN,14.0,500000,Free,0.0,Everyone,Art & Design;Pretend Play,4
2,ART_AND_DESIGN,8.7,5000000,Free,0.0,Everyone,Art & Design,4


In [46]:
df.to_csv(FILE_OUT, index=False)
print("Saved:", FILE_OUT)


Saved: googleplaystore_clean.csv
